In [1]:
import contextlib
from time import sleep

import qibo
from qibo import gates
from qibo.core.circuit import Circuit

from qiboconnection.api import API
from qiboconnection.connection import ConnectionConfiguration

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:56]: Qibo Quantum Service API SERVER URL: https://qilimanjaro.ddns.net:8080
[Qibo 0.1.6|INFO|2022-03-10 09:56:56]: Using qibojit backend on /CPU:0


## Connect to the Qibo Service API

In [2]:
myconf = ConnectionConfiguration(user_id=1, username="my-user-name", api_key="abcdefg-hijk-lmno-pqrs-tuvwxyzABCDE")
qibo_api = API(configuration=myconf)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:56]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/authorisation-tokens
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:56]: Connection successfully established.
[qibo-connection] 0.0.2|INFO|2022-03-10 09:56:56]: Storing personal qibo configuration...
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:56]: Configuration loaded successfully.


## Load an already existing connection to the Qilimanjaro Service API

In [ ]:
qibo_api = API()

## Check a connection is alive

In [ ]:
qibo_api.ping()

## List all registered devices

In [3]:
devices = qibo_api.list_devices()
devices

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:56]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices


<Devices[4]:
{
  "device_id": 1,
  "device_name": "Radagat simulator",
  "status": "available",
  "characteristics": {
    "type": "simulator",
    "cpu": "Intel Core i9-9900K @ 16x 5GHz",
    "gpu": "NVIDIA GeForce RTX 3090",
    "os": "Ubuntu 20.04 focal",
    "kernel": "x86_64 Linux 5.4.0-80-generic",
    "ram": "64185MiB"
  }
}
{
  "device_id": 9,
  "device_name": "Galadriel Qblox rack",
  "status": "available",
  "characteristics": {
    "type": "quantum"
  },
  "calibration_details": {
    "t1": 12,
    "frequency": 988
  }
}
{
  "device_id": 10,
  "device_name": "Galadriel Qblock cluster",
  "status": "available",
  "characteristics": {
    "type": "quantum"
  },
  "calibration_details": {
    "t1": 12,
    "frequency": 988
  }
}
{
  "device_id": 11,
  "device_name": "Offline Test                                      ",
  "status": "offline"
}

## Selecting a specific device

In [5]:
qibo_api.select_device_id(device_id=9)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:57]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/9
[qibo-connection] 0.0.2|INFO|2022-03-10 09:56:58]: Device Galadriel Qblox rack selected.


## Creating an experiment to be executed on a remote device

In [6]:
qibo.set_backend("numpy")

[Qibo 0.1.6|INFO|2022-03-10 09:56:58]: Using numpy backend on /CPU:0


In [7]:
circuit = Circuit(1)
circuit.add(gates.H(0))
circuit.add(gates.M(0))

## Remote execution and getting a job identifier

In [8]:
job_id = qibo_api.execute(circuit=circuit)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:58]: Sending qibo circuit for a remote execution...
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:58]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/circuits
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:58]: Job circuit queued successfully.


In [9]:
job_id

2828

## User retrieves the Qibo result, but it is still pending

In [10]:
result = qibo_api.get_result(job_id=job_id)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:56:59]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/jobs/2828
[qibo-connection] 0.0.2|INFO|2022-03-10 09:56:59]: Your job is still pending. Job queue position: 1


## When job is executed remotely, a user can retrieve the actual result

In [11]:
sleep(1)
result = qibo_api.get_result(job_id=job_id)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:00]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/jobs/2828
[qibo-connection] 0.0.2|INFO|2022-03-10 09:57:00]: Your job is completed.


In [12]:
result.state()

array([0.70710678+0.j, 0.70710678+0.j])

In [13]:
result.frequencies(binary=True, registers=True)

{'register0': Counter({'0': 6, '1': 4})}

In [14]:
circuit.draw()

'q0: ─H─M─'

In [15]:
qibo_api.release_device(device_id=9)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:00]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/9
[qibo-connection] 0.0.2|INFO|2022-03-10 09:57:01]: Device Galadriel Qblox rack released.


In [16]:
devices._devices[0].id

1

## Block and release a remote device to operate on it

In [17]:
qibo_api.block_device_id(device_id=1)

# Do stuff with the device, knowing no one else will access it

qibo_api.release_device(device_id=1)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:01]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:01]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|INFO|2022-03-10 09:57:01]: Device Radagat simulator blocked.
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:01]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|INFO|2022-03-10 09:57:01]: Device Radagat simulator released.


## Trying to block a blocked device will raise an exception

In [18]:
with contextlib.suppress(Exception):
    qibo_api.block_device_id(device_id=1)
    qibo_api.block_device_id(device_id=1)

qibo_api.release_device(device_id=1)

[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:02]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:02]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|INFO|2022-03-10 09:57:02]: Device Radagat simulator blocked.
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:02]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|DEBUG|2022-03-10 09:57:02]: Calling: https://qilimanjaro.ddns.net:8080/api/v1/devices/1
[qibo-connection] 0.0.2|ERROR|2022-03-10 09:57:02]: {
  "title": "Bad Request",
  "status": 400,
  "detail": "Device Radagat simulator is already busy. 400 Client Error: BAD REQUEST for url: https://qilimanjaro.ddns.net:8080/api/v1/devices/1"
}
[qibo-connection] 0.0.2|ERROR|2022-03-10 09:57:02]: {
  "title": "Bad Request", 
  "status": 400, 
  "detail": "Device Radagat simulator is already busy."
}

[qibo-connection] 0.0.2|ERROR|2022-03-10 09:57:02]: Error blocki